<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_02/TS%20-%20W2%20-%2008%20-%20Rede%20neural%20de%20camada%20%C3%BAnica%20(Laborat%C3%B3rio%202).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Treinamento de uma rede neural de camada única com dados de séries temporais

Agora que você já viu os métodos estatísticos na semana anterior, passará a usar as redes neurais para criar seus modelos de previsão.

Você começará com uma rede simples neste notebook e nos próximos passará para arquiteturas mais complexas.

Ao final deste laboratório, você será capaz de:

* criar uma rede de camada única e treiná-la usando os mesmos dados sintéticos que você usou no laboratório anterior
* preparar dados de séries temporais para treinamento e avaliação
* medir o desempenho do seu modelo em relação a um conjunto de validação

## Importações

Primeiro, você importará os pacotes de que precisará para executar todo o código deste laboratório. Você usará:
* [Tensorflow](https://www.tensorflow.org/api_docs/python/tf) para criar seu modelo e preparar janelas de dados
* [Numpy](https://numpy.org/) para processamento numérico
* e a biblioteca [PyPlot](https://matplotlib.org/3.5.1/api/_as_gen/matplotlib.pyplot.html) do Matplotlib para visualização

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Utilitários

Em seguida, você definirá algumas funções utilitárias que também viu nos laboratórios anteriores.

Elas se encarregarão da visualização dos dados da série temporal e as previsões do modelo, além de gerar os dados sintéticos.

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    """
    Visualizar dados de séries temporais

    Args:
      time (vetor de int) - contém as etapas de tempo
      series (vetor de int) - contém as medidas para cada etapa de tempo
      format - estilo da linha ao plotar o gráfico
      label - rótulo para a linha
      start - primeiro passo de tempo a ser plotado
      end - última etapa de tempo a ser plotada
    """

    # Configuração das dimensões da figura do gráfico
    plt.figure(figsize=(10, 6))

    if type(series) is tuple:

      for series_num in series:
        # Plotar os dados da série temporal
        plt.plot(time[start:end], series_num[start:end], format)

    else:
      # Plotar os dados da série temporal
      plt.plot(time[start:end], series[start:end], format)

    # Rotular o eixo x
    plt.xlabel("Time")

    # Rotular o eixo y
    plt.ylabel("Value")

    # Sobrepor uma grade no gráfico
    plt.grid(True)

    # Desenhar o gráfico na tela
    plt.show()


def trend(time, slope=0):
    """
    Gera dados sintéticos que seguem uma linha reta com um valor de inclinação.

    Args:
      time (vetor de int) - contém as etapas de tempo
      slope (float) - determina a direção e a inclinação da linha

    Retorna:
      series (vetor de float) - medições que seguem uma linha reta
    """

    # Calcula a série linear dada a inclinação
    series = slope * time

    return series


def seasonal_pattern(season_time):
    """
    Apenas um padrão arbitrário, você pode alterá-lo se desejar

    Args:
      season_time (vetor de float) - contém as medições por etapa de tempo

    Retorna:
      data_pattern (vetor de float) - contém os valores de medição revisados de acordo com
                                      o padrão definido
    """

    # Gerar os valores usando um padrão arbitrário
    data_pattern = np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

    return data_pattern


def seasonality(time, period, amplitude=1, phase=0):
    """
    Repete o mesmo padrão em cada período

    Args:
      time (vetor de int) - contém as etapas de tempo
      period (int) - número de etapas de tempo antes da repetição do padrão
      amplitude (int) - valor de pico medido em um período
      phase (int) - número de etapas de tempo para deslocar os valores medidos

    Retorna:
      data_pattern (vetor de float) - dados sazonais dimensionados pela amplitude definida
    """

    # Definir os valores medidos por período
    season_time = ((time + phase) % period) / period

    # Gera os dados sazonais dimensionados pela amplitude definida
    data_pattern = amplitude * seasonal_pattern(season_time)

    return data_pattern


def noise(time, noise_level=1, seed=None):
    """Gera um sinal ruidoso normalmente distribuído

    Args:
      time (vetor de int) - contém as etapas de tempo
      noise_level (float) - fator de escala para o sinal gerado
      seed (int) - semente do gerador de números para repetibilidade

    Retorna:
      noise (vetor de float) - o sinal ruidoso
    """

    # Inicializar o gerador de números aleatórios
    rnd = np.random.RandomState(seed)

    # Gerar um número aleatório para cada etapa de tempo e dimensionar pelo nível de ruído
    noise = rnd.randn(len(time)) * noise_level

    return noise

## Gerar os dados sintéticos

O código abaixo gera os mesmos dados sintéticos que você usou no laboratório anterior.

Ele conterá 1.461 pontos de dados com:
* tendência,
* sazonalidade e
* ruído.

In [ ]:
# Parâmetros
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Criar a série
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)

# Atualizar com ruído
series += noise(time, noise_level, seed=42)

# Plotar os resultados
plot_series(time, series)

## Dividir o conjunto de dados

Em seguida, você dividirá os dados acima em conjuntos de treinamento e validação. Você pegará os primeiros 1.000 pontos para treinamento, enquanto o restante será para validação,

In [ ]:
# Definir o tempo de divisão
split_time = 1000

# Obter o conjunto de treino
time_train = time[:split_time]
x_train = series[:split_time]

# Obter o conjunto de validação
time_valid = time[split_time:]
x_valid = series[split_time:]

Você pode inspecionar esses conjuntos visualmente usando a mesma função de utilidade para plotagem.

Observe que, em geral, o conjunto de validação tem valores mais altos (ou seja, o eixo y) do que os do conjunto de treinamento.

Seu modelo deve ser capaz de prever esses valores apenas aprendendo com a tendência e a sazonalidade do conjunto de treinamento.

In [ ]:
# Plotar o conjunto de treino
plot_series(time_train, x_train)

In [ ]:
# Plotar o conjunto de validação
plot_series(time_valid, x_valid)

## Preparar recursos e rótulos

Em seguida, você preparará as janelas de dados conforme mostrado no laboratório anterior.

É bom declarar os parâmetros em uma célula separada para que você possa ajustá-los facilmente mais tarde, se desejar.

In [ ]:
# Hiperparâmetros da série temporal
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

A função a seguir contém todas as etapas de pré-processamento que você realizou no laboratório anterior.
> Isso a torna modular para que você possa usá-la facilmente em seus outros projetos, se necessário.

Um aspecto a ser observado aqui é o `window_size + 1` quando você chama `dataset.window()`.
* Há um `+ 1` para indicar que você está usando o próximo ponto como rótulo.
* Por exemplo, os primeiros 20 pontos serão o recurso, portanto, o 21º ponto será o rótulo.

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Gera janelas de conjunto de dados

    Args:
      series (vetor of float) - contém os valores da série temporal
      window_size (int) - o número de etapas de tempo a serem incluídas no recurso
      batch_size (int) - o tamanho do lote
      shuffle_buffer(int) - tamanho do buffer a ser usado para o método shuffle

    Retorna:
      dataset (TF Dataset) - Conjunto de dados TF contendo janelas de tempo
    """

    # Gerar um conjunto de dados TF a partir dos valores da série
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Janela de dados, mas só pega aqueles com o tamanho especificado
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)

    # Achatar as janelas, colocando seus elementos em um único lote
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Criar tuplas com recursos e rótulos
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Embaralhar as janelas
    dataset = dataset.shuffle(shuffle_buffer)

    # Criar lotes de janelas
    dataset = dataset.batch(batch_size).prefetch(1)

    return dataset

Agora você pode gerar as janelas do conjunto de dados a partir do conjunto de treinamento.

In [ ]:
# Gerar as janelas do conjunto de dados
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

Você pode inspecionar novamente a saída para ver se a função está se comportando conforme o esperado.

O código abaixo usará o método [`take()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take) da API `tf.data.Dataset` para obter um único lote.
> Em seguida, ele imprimirá várias propriedades desse lote, como o tipo de dados e a forma dos elementos.

Como esperado, ele deve ter uma tupla de 2 elementos (ou seja, `(feature, label)`) e as formas desses elementos devem estar alinhadas com os tamanhos de lote e de janela que você declarou anteriormente, que são `32` e `20` por padrão, respectivamente.

In [ ]:
# Imprimir propriedades de um único lote
for windows in dataset.take(1):
  print(f'tipo de dados: {type(windows)}')
  print(f'número de elementos na tupla: {len(windows)}')
  print(f'forma do primeiro elemento: {windows[0].shape}')
  print(f'forma do segundo elemento: {windows[1].shape}')

## Criar e compilar o modelo

Em seguida, você criará a rede neural de camada única.

Será apenas uma camada [Dense](https://keras.io/api/layers/core_layers/dense/) de uma unidade, conforme mostrado abaixo.

Você atribuirá a camada a uma variável `l0` para que também possa ver os pesos finais posteriormente usando o método [`get_weights()`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#get_weights).

In [ ]:
# Construir a rede neural de camada única
l0 = tf.keras.layers.Dense(1, input_shape=[window_size])
model = tf.keras.models.Sequential([l0])

# Imprimir os pesos da camada inicial
print("Pesos da camada: \n {} \n".format(l0.get_weights()))

# Imprimir o resumo do modelo
model.summary()

Você definirá [*mean squared error (mse)*](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError) como a função de perda e usará [*stochastic gradient descent (SGD)*](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD) para otimizar os pesos durante o treinamento.

In [ ]:
# Definir os parâmetros de treinamento
model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9))

## Treinar o modelo

Agora você pode prosseguir com o treinamento do modelo. Você alimentará as janelas de dados preparadas e executará o treinamento por 100 épocas.

In [ ]:
# Treine o modelo
model.fit(dataset,epochs=100)

Você pode ver os pesos finais chamando novamente o método `get_weights()`.

In [ ]:
# Print the layer weights
print("Layer weights {}".format(l0.get_weights()))

## Previsão do modelo

Com o treinamento concluído, agora você pode medir o desempenho do seu modelo.

Você pode gerar uma previsão de modelo passando um lote de janelas de dados.

Se você for cortar uma janela da matriz `series` original, precisará adicionar uma dimensão de lote antes de passá-la para o modelo.

Isso pode ser feito indexando com a constante [`np.newaxis`](https://numpy.org/doc/stable/reference/constants.html?highlight=newaxis#numpy.newaxis) ou usando o método [`np.expand_dims()`](https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html).

In [ ]:
# Forma dos primeiros 20 pontos de dados slice
print(f'formato da série[0:20]: {series[0:20].shape}')

# Forma após adicionar uma dimensão de lote
print(f'formato da série[0:20][np.newaxis]: {series[0:20][np.newaxis].shape}')

# Forma após adicionar uma dimensão de lote (forma alternativa)
print(f'formato da série[0:20][np.newaxis]: {np.expand_dims(series[0:20], axis=0).shape}')

# Previsão de modelo de amostra
print(f'predição do modelo: {model.predict(series[0:20][np.newaxis])}')

Para calcular as métricas, você deverá gerar previsões de modelos para seu conjunto de validação.
* Lembre-se de que esse conjunto se refere a pontos nos índices `1000` a `1460` de toda a série.
* Você precisará codificar as etapas para gerar essas previsões a partir do seu modelo. A célula abaixo demonstra uma maneira de fazer isso.

Basicamente, ela alimenta o modelo com toda a série, 20 pontos de cada vez, e anexa todos os resultados a uma lista `forecast`. Em seguida, ele cortará os pontos que correspondem ao conjunto de validação.

O índice de fatia abaixo é `split_time - window_size:` porque a lista de previsão é menor do que a série em 20 pontos (ou seja, o tamanho da janela).
* Como o tamanho da janela é 20, o primeiro ponto de dados na lista `forecast` corresponde à previsão de tempo no índice `20`.
* Você não pode fazer previsões nos índices `0` a `19` porque eles são menores que o tamanho da janela.
* Portanto, ao dividir com `split_time - window_size:`, você obterá os pontos nos índices de tempo que se alinham com os do conjunto de validação.

**Observação: você pode notar que essa célula demora um pouco para ser executada. Nos próximos dois laboratórios, você verá outras abordagens para gerar previsões e acelerar a execução do código. Talvez você já tenha algumas ideias e fique à vontade para testá-las depois de concluir este laboratório.**

In [ ]:
# Inicializar uma lista
forecast = []

# Use o modelo para prever pontos de dados por tamanho de janela
for time in range(len(series) - window_size):
  forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

# Corte os pontos que estão alinhados com o conjunto de validação
forecast = forecast[split_time - window_size:]

# Compare o número de elementos nas previsões e no conjunto de validação
print(f'comprimento da lista de previsão: {len(forecast)}')
print(f'forma do conjunto de validação: {x_valid.shape}')

Para visualizar os resultados, você precisará converter as previsões em um formato aceito pela função do utilitário `plot_series()`.

Isso envolve a conversão da lista em uma matriz numérica e a eliminação dos eixos unidimensionais.

In [ ]:
# Visualizar formas depois de usar os métodos de conversão e compressão
print(f'forma após a conversão para matriz numérica: {np.array(forecast).shape}')
print(f'formato após a compressão: {np.array(forecast).squeeze().shape}')

# Converta em uma matriz numérica e elimine os eixos de dimensão única
results = np.array(forecast).squeeze()

# Sobrepor os resultados com o conjunto de validação
plot_series(time_valid, (x_valid, results))

Você pode calcular as métricas chamando as mesmas funções de antes. Você obterá um MAE próximo a 5.

In [ ]:
# Calcular as métricas
print(tf.keras.metrics.mean_squared_error(x_valid, results).numpy())
print(tf.keras.metrics.mean_absolute_error(x_valid, results).numpy())

## Resumo

Neste laboratório, você conseguiu criar e treinar uma rede neural de camada única em dados de séries temporais:
* preparou janelas de dados
* alimentou o modelo e
* as previsões finais mostram resultados comparáveis com a análise estatística que você anteiormente.

Nos próximos laboratórios, você tentará adicionar mais camadas e também verá algumas otimizações que podem ser feitas ao treinar o modelo.